# Topic modelling - LDA

## Import libraries 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")

In [3]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction import text

In [1]:
# Libraries to connect to oracle database
import cx_Oracle as orc
import os
from sqlalchemy import types, create_engine

## Import data

In [13]:
# Make DSN address
dsn = orc.makedsn('hostname', 'port', service_name='servicename')
# Connect to DSN
conn = orc.connect(user='username', password='password', dsn=dsn)

In [14]:
sql_query = """SELECT 
  RESPONSE_ID,
  DATE_SUBMITTED,
  NPS_RATING,
  NPS_COMMENT,
  FEEDBACK
FROM
  "DATABASE_NAME"."TABLE_NAME"
WHERE
  DATE_SUBMITTED >= TO_DATE('01.11.18', 'DD.MM.YY') AND
  DATE_SUBMITTED <= TO_DATE('30.11.18', 'DD.MM.YY')"""

In [15]:
# Read database table into dataframe 
original_text_df = pd.read_sql(sql_query, conn)

In [16]:
text_df = original_text_df.copy()

In [17]:
text_df.head()

,RESPONSE_ID,DATE_SUBMITTED,NPS_RATING,NPS_COMMENT,FEEDBACK
0,133294,2018-01-11 00:01:35,10.0,NaN,N / A
1,133295,2018-01-11 13:19:07,8.0,NaN,N / A
2,133296,2018-01-11 00:40:47,8.0,NaN,N / A
3,133297,2018-01-11 00:41:36,4.0,keine Lieferkosten,no delivery costs
4,133298,2018-01-11 00:42:42,NaN,NaN,N / A


In [19]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32623 entries, 0 to 32622
Data columns (total 5 columns):
RESPONSE_ID       32623 non-null int64
DATE_SUBMITTED    32623 non-null datetime64[ns]
NPS_RATING        29193 non-null float64
NPS_COMMENT       6254 non-null object
FEEDBACK          32614 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 1.2+ MB


Among the 32623 feedback present in the dataset, only 6254 feedback has text comment in it

In [20]:
text_df = text_df.drop(['RESPONSE_ID'], axis=1)

In [21]:
text_df['DATE_SUBMITTED'] = text_df['DATE_SUBMITTED'].dt.date

## Retain feedback that contain text data 

In [23]:
text_df = text_df.dropna(subset=['FEEDBACK'])

In [24]:
text_df = text_df[text_df['FEEDBACK']!='N / A']

In [25]:
pd.set_option("display.max_colwidth", 300)
text_df.tail(5)

,DATE_SUBMITTED,NPS_RATING,NPS_COMMENT,FEEDBACK
32603,2018-11-30,6.0,"wenn ich im Pax Planer eine Bestellung aufgeben mÃ¶chte, ist der Ãbergang zum Warenkorb sehr nervenaufreibend. ich habe mehrere AnlÃ¤ufe gebraucht, bis es geklappt hat. - bei mehreren Aktionen im Pax Planer ist die Ãbersicht sehr undeutlich, welche Artikel nun im Warenkorb sind und welche nich...","If I want to place an order in the Pax Planner, the transition to shopping cart is very nerve-wracking. It took me several tries to get it to work. - With several actions in the Pax Planner the overview is very unclear, which articles are in the shopping cart and which are not. the colors for th..."
32604,2018-11-30,8.0,Eine kleinschrittigere Kategorisierung,A smaller step categorization
32610,2018-11-30,NaN,bis jetzt zufrieden í ½í¸,satisfied until now
32613,2018-11-30,8.0,"Kauf per PayPal ging nicht, lÃ¤dt wahrscheinlich immer noch","Purchase via PayPal did not work, probably still loads"
32617,2018-11-30,7.0,"Bezahlmethoden, bei Klarna ist es wirklich sehr umstÃ¤ndlich auf den Geburtstag zu kommen. Das macht keinen SpaÃ.","Payment methods, at Klarna it is really very awkward to come on the birthday. That&#39;s not fun."


In [35]:
text_df1 = text_df.copy()

## Remove punctuation and conversion to lowercase

In [36]:
text_df1['FEEDBACK_PROCESSED'] = text_df1['FEEDBACK'].map(lambda x: re.sub('[,\.!?]', '', x))
text_df1['FEEDBACK_PROCESSED'] = text_df1['FEEDBACK_PROCESSED'].map(lambda x: x.lower())
text_df1['FEEDBACK_PROCESSED'].head(10)

3                                                                                                                              no delivery costs
5                                                                                      if more items could be bought online i would buy more :-)
6                                                                                                                                    the clarity
11                                                                                                                                     all right
13                                                                                                                         shorter delivery time
15                                                                                                                more and faster delivery times
20                                                                               other payment options such as sofortüberweisung o

## Transform text data using CountVectorizer

In [37]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(['quot', '39'])

In [38]:
count_vectorizer = CountVectorizer(max_df=0.8, min_df=2, stop_words=my_stop_words)

In [39]:
count_data = count_vectorizer.fit_transform(text_df1['FEEDBACK_PROCESSED'])

## LDA

In [40]:
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(", ".join([words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [41]:
number_topics = 10
number_words = 10

lda = LDA(n_components=number_topics,n_jobs =-1)
lda.fit(count_data)
print_topics(lda, count_vectorizer, number_words)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)



Topic #0:
clear, perfect, options, payment, direct, need, furniture, debit, search, white

Topic #1:
work, search, did, ok, better, does, paypal, function, forwarding, different

Topic #2:
payment, card, account, ikea, family, number, complicated, credit, code, telephone

Topic #3:
great, cart, shopping, date, pax, planner, click, enter, order, collect

Topic #4:
online, shop, available, products, product, ikea, articles, article, ordered, nice

Topic #5:
delivery, time, long, pay, order, service, customer, purchase, ordering, possible

Topic #6:
just, order, think, like, make, buy, online, offer, better, directly

Topic #7:
shipping, costs, delivery, free, high, order, cheaper, value, small, bit

Topic #8:
items, order, ikea, store, online, furniture, far, expensive, wanted, transfer

Topic #9:
delivery, good, faster, times, goods, time, bad, satisfied, easy, displayed


### Finding the highest probable topic from the feedback 

In [42]:
topic_values = lda.transform(count_data)
text_df1['HIGHEST_TOPIC'] = topic_values.argmax(axis=1)
text_df1.head()

,DATE_SUBMITTED,NPS_RATING,NPS_COMMENT,FEEDBACK,FEEDBACK_PROCESSED,HIGHEST_TOPIC
3,2018-01-11,4.0,keine Lieferkosten,no delivery costs,no delivery costs,7
5,2018-01-11,10.0,Wenn mehr Artikel online gekauft werden kÃ¶nnten. Ich wÃ¼rde noch mehr kaufen :-),If more items could be bought online. I would buy more :-),if more items could be bought online i would buy more :-),4
6,2018-01-11,9.0,die Ãbersichtlichkeit,the clarity,the clarity,1
11,2018-01-11,10.0,alles toll,All right,all right,5
13,2018-01-11,NaN,KÃ¼rzere lieferzeit,Shorter delivery time,shorter delivery time,9


### Finding topics with probability of more than 0.3 for each of the feedback and attaching it to the original dataframe. This is done since one feedback could have multiple topics disussed in it. 

In [43]:
text_df2 = text_df1.copy()

In [44]:
text_df2.head()

,DATE_SUBMITTED,NPS_RATING,NPS_COMMENT,FEEDBACK,FEEDBACK_PROCESSED,HIGHEST_TOPIC
3,2018-01-11,4.0,keine Lieferkosten,no delivery costs,no delivery costs,7
5,2018-01-11,10.0,Wenn mehr Artikel online gekauft werden kÃ¶nnten. Ich wÃ¼rde noch mehr kaufen :-),If more items could be bought online. I would buy more :-),if more items could be bought online i would buy more :-),4
6,2018-01-11,9.0,die Ãbersichtlichkeit,the clarity,the clarity,1
11,2018-01-11,10.0,alles toll,All right,all right,5
13,2018-01-11,NaN,KÃ¼rzere lieferzeit,Shorter delivery time,shorter delivery time,9


In [45]:
text_df2 = text_df2.reset_index()

In [46]:
text_df2['ROW_NO'] = text_df2.index + 0

In [47]:
df_topicmodel = pd.DataFrame(np.argwhere(topic_values>0.3))

In [48]:
df_topicmodel.head(5)

,0,1
0,0,7
1,1,4
2,2,1
3,3,5
4,4,9


In [49]:
df_topicmodel.rename(columns={0:'ROW_NO', 1:'TOPICS'}, inplace=True)

In [50]:
text_df3 = pd.merge(text_df2, df_topicmodel, on='ROW_NO', how='left')

In [51]:
text_df3.head()

,index,DATE_SUBMITTED,NPS_RATING,NPS_COMMENT,FEEDBACK,FEEDBACK_PROCESSED,HIGHEST_TOPIC,ROW_NO,TOPICS
0,3,2018-01-11,4.0,keine Lieferkosten,no delivery costs,no delivery costs,7,0,7.0
1,5,2018-01-11,10.0,Wenn mehr Artikel online gekauft werden kÃ¶nnten. Ich wÃ¼rde noch mehr kaufen :-),If more items could be bought online. I would buy more :-),if more items could be bought online i would buy more :-),4,1,4.0
2,6,2018-01-11,9.0,die Ãbersichtlichkeit,the clarity,the clarity,1,2,1.0
3,11,2018-01-11,10.0,alles toll,All right,all right,5,3,5.0
4,13,2018-01-11,NaN,KÃ¼rzere lieferzeit,Shorter delivery time,shorter delivery time,9,4,9.0


In [52]:
text_df3 = text_df3.drop('index', axis=1)

In [155]:
# Defining names for each of the topics
text_df3.loc[text_df3['TOPICS'] == 0.0, 'TOPICS_NAME'] = 'abc'
text_df3.loc[text_df3['TOPICS'] == 1.0, 'TOPICS_NAME'] = 'abc'
text_df3.loc[text_df3['TOPICS'] == 2.0, 'TOPICS_NAME'] = 'abc'
text_df3.loc[text_df3['TOPICS'] == 3.0, 'TOPICS_NAME'] = 'abc'
text_df3.loc[text_df3['TOPICS'] == 4.0, 'TOPICS_NAME'] = 'abc'
text_df3.loc[text_df3['TOPICS'] == 5.0, 'TOPICS_NAME'] = 'abc'
text_df3.loc[text_df3['TOPICS'] == 6.0, 'TOPICS_NAME'] = 'abc'
text_df3.loc[text_df3['TOPICS'] == 7.0, 'TOPICS_NAME'] = 'abc'
text_df3.loc[text_df3['TOPICS'] == 8.0, 'TOPICS_NAME'] = 'abc'
text_df3.loc[text_df3['TOPICS'] == 9.0, 'TOPICS_NAME'] = 'abc'